# Informarion Retrival Project : Build an Intelligent Information Retrival System

## Importation des librairies

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import json

In [2]:
documents = open("../cisi/CISI.ALL").read()
queries = open("../cisi/CISI.QRY").read()
documents[:1000], queries[:1000]

(".I 1\n.T\n18 Editions of the Dewey Decimal Classifications\n.A\nComaromi, J.P.\n.W\n   The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.\n.X\n1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1\n.I 2\n.T \nUse Made of Technical Libraries\n.A \nSlater, M.\n.W\nThis report is an analysis of 6300 acts of use\nin 104 technical libraries in the United Kingdom.\nLibrary use is only one aspect of the wider pattern of\ninformation use.  Information transfer in libraries is\nrestrict

## Indexiation des des documents

### Extraire les termes avec split
### Extraire les termes en utilisant la tokenization
### Supprimer les mots vides
### Normaliser les terme
### Pondérer les termes

In [3]:
def split_extraction(documents):
    content = documents.split(".I ")
    del content[0]

    parts = content
    data = []
    for part in parts:
        
        temp = part.split(".T\n")
        if len(temp) == 1 : temp = part.split(".T \n")
        if len(temp) == 1 : temp = part.split(".T  \n")
        
        del temp[0]
        aut_temp = temp[0].split(".A\n") if len(temp[0].split(".A\n")) >= 2 else temp[0].split(".A \n")
        if len(aut_temp) > 2 : 
            while len(aut_temp) != 2:
                del aut_temp[1]
        title, temp2 = aut_temp
        
        summary = temp2.split(".W")[1].split(".X")[0]
        data.append([' '.join(title.split()).lower(),' '.join(summary.split()).lower()])
    return data
data = split_extraction(documents)
pd.DataFrame(data, columns = ["Titre","Résumé"])

,Titre,Résumé
0,18 editions of the dewey decimal classifications,the present study is a history of the dewey de...
1,use made of technical libraries,this report is an analysis of 6300 acts of use...
2,two kinds of power an essay on bibliographic c...,the relationships between the organization and...
3,systems analysis of a university library; fina...,the establishment of nine new universities in ...
4,a library management game: a report on a resea...,although the use of games in professional educ...
...,...,...
1455,world dynamics,.
1456,world trends in library education,one of the most significant aspects of the evo...
1457,legal restrictions on exploitation of the pate...,the patent laws confer on a patentee power to ...
1458,language and thought,this book considers the basic aspects of this ...


In [4]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [5]:
def get_documents_inverse(data, stem):
    i = 1
    documents_inverse = []
    for d in data:
        text = ""
        text += ' '.join(d) + " "
        rgx = '(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%? |\w+(?:\-\w+)*'
        text_token = nltk.RegexpTokenizer(rgx).tokenize(text)
        mots_vides = nltk.corpus.stopwords.words('english')
        termes_sans_mots_vides = [terme for terme in text_token if terme.lower() not in mots_vides]
        ps = nltk.PorterStemmer()
        ls = nltk.LancasterStemmer()
        if stem == "p" : stemmed = [ps.stem(tt) for tt in termes_sans_mots_vides]
        elif stem == "l" : stemmed = [ls.stem(tt) for tt in termes_sans_mots_vides]
        freq_word_doc = nltk.FreqDist(stemmed)

        filtered_word_freq = [[word, "D"+str(i), freq] 
                                for word, freq in freq_word_doc.items() if not word.isdigit()]
        i += 1
        documents_inverse.append(filtered_word_freq)

    return pd.DataFrame(flatten(documents_inverse), columns = ["Word","Document","Frequence"])
document_inverse = get_documents_inverse(data, "p")
document_inverse

,Word,Document,Frequence
0,18,D1,1
1,edit,D1,4
2,dewey,D1,3
3,decim,D1,2
4,classif,D1,2
...,...,...,...
73639,9%,D1460,1
73640,100-150,D1460,1
73641,thousand,D1460,1
73642,new,D1460,1


In [6]:
dictionnaire_inverse = document_inverse.groupby('Document')[['Word','Frequence']].apply(lambda g: g.values.tolist()).to_dict()

with open('../inverse.json', 'w') as fp:
    json.dump(dictionnaire_inverse, fp)

In [7]:
def ponderation(document_inverse):
    max_freq = document_inverse.groupby('Document')['Frequence'].max().to_frame()
    max_freq.reset_index(inplace=True)
    max_freq.rename(columns = {'Frequence':'Max Frequence'}, inplace = True)
    df = document_inverse.merge(max_freq, on="Document")

    nb_doc = document_inverse.groupby('Word')['Document'].count().to_frame()
    nb_doc.reset_index(inplace=True)
    nb_doc.rename(columns = {'Document':'Nombre document contenu'}, inplace = True)
    df = df.merge(nb_doc, on="Word")

    df["Poid"] = np.multiply(
        np.divide(
            df["Frequence"], 
            df["Max Frequence"]
            ), 
            np.log10(
                np.divide(
                    len(document_inverse['Document'].unique()), 
                    df["Nombre document contenu"]
                ) + 1
            )
        )

    fichier_descripteur = df[["Document", "Word", "Frequence", "Poid"]].sort_values(by=["Document"])
    fichier_inverse = df[["Word", "Document", "Frequence", "Poid"]].sort_values(by=["Word"])

    return fichier_descripteur, fichier_inverse

fichier_descripteur, fichier_inverse = ponderation(document_inverse)


In [8]:
fichier_descripteur

,Document,Word,Frequence,Poid
0,D1,18,1,0.596996
1858,D1,told,1,0.640870
1843,D1,never,1,0.498175
1839,D1,stori,1,0.640870
1806,D1,full,1,0.413886
...,...,...,...,...
3100,D999,detail,1,0.301379
388,D999,present,2,0.385401
2523,D999,system,1,0.147678
52837,D999,data,1,0.196738


#### Retourner la liste des termes d'un document donné (avec fréquences et poids)

In [9]:
list_of_terms = fichier_descripteur.loc[fichier_descripteur["Document"] == "D2"]
list_of_terms[["Word", "Frequence", "Poid"]]

,Word,Frequence,Poid
5872,aspect,1,0.150168
5027,report,1,0.119946
7130,outsid,1,0.236402
7458,doubt,1,0.261091
8546,handbook,1,0.270915
7309,channel,1,0.236402
5970,wider,1,0.249088
5985,pattern,1,0.158046
9343,receiv,1,0.187145
7604,user,2,0.219445


In [10]:
fichier_inverse

,Word,Document,Frequence,Poid
72708,0.18,D1090,1,0.527442
71056,0.7%,D691,1,0.395581
58996,000,D403,1,0.413886
58997,000,D442,1,0.331109
58998,000,D463,1,0.827773
...,...,...,...,...
67925,zipfian,D329,1,0.527442
56806,zone,D62,2,0.575391
71321,zoolog,D755,1,0.632930
73255,zuckerman,D1291,1,1.431959


#### Retourner la liste des documents contenant un terme donné (avec fréquence et poids)

In [11]:
list_of_terms = fichier_inverse.loc[fichier_inverse["Word"] == "class"]
list_of_terms[["Document", "Frequence", "Poid"]]

,Document,Frequence,Poid
17598,D722,1,0.189383
17572,D5,1,0.252511
17593,D559,1,0.216438
17592,D486,1,0.303013
17591,D479,3,0.649314
17590,D478,1,0.252511
17589,D476,1,0.505022
17588,D455,1,0.216438
17587,D417,1,0.303013
17573,D16,1,0.378767


## Indexation des requetes

In [12]:
def extract_queries(queries):
    content = queries.split(".I ")
    del content[0]
    parts = content
    data = []
    for part in parts:
        if ".T" in part:
            temp = part.split(".T\n")
            del temp[0]
            aut_temp = temp[0].split(".A\n")
            title, temp2 = aut_temp
            
            query = temp2.split(".W\n")[1].split(".B\n")[0]
            data.append([' '.join(title.split()).lower(),' '.join(query.split()).lower()])
        else:
            query = part.split(".W\n")[1]
            data.append(['',' '.join(query.split()).lower()])
    return data

data_queries = extract_queries(queries)
pd.DataFrame(data_queries, columns = ["Titre","Query"])

,Titre,Query
0,,what problems and concerns are there in making...
1,,"how can actually pertinent data, as opposed to..."
2,,what is information science? give definitions ...
3,,image recognition and any other methods of aut...
4,,what special training will ordinary researcher...
...,...,...
107,a program for machine-mediated searching,a technique of online instruction and assistan...
108,author cocitation: a literature measure of int...,it is shown that the mapping of a particular a...
109,progress in documentation. word processing: an...,"the ""office of the future,"" ""office technology..."
110,document clustering using an inverted file app...,an automated document clustering procedure is ...


In [13]:
def format_queries(data, stem):
    i = 1
    info_queries = []
    for d in data:
        text = ""
        if len(d[0]) != 0: text += ' '.join(d) + " "
        else : text += ''.join(d) + " "
        rgx = '(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%? |\w+(?:\-\w+)*'
        text_token = nltk.RegexpTokenizer(rgx).tokenize(text)
        mots_vides = nltk.corpus.stopwords.words('english')
        termes_sans_mots_vides = [terme for terme in text_token if terme.lower() not in mots_vides]
        ps = nltk.PorterStemmer()
        ls = nltk.LancasterStemmer()
        if stem == "p": stemmed = [ps.stem(tt) for tt in termes_sans_mots_vides]
        elif stem == "l": stemmed = [ls.stem(tt) for tt in termes_sans_mots_vides]
        filtered_word_freq = [["Q"+str(i), word] for word in stemmed if not word.isdigit()]
        i += 1
        info_queries.append(filtered_word_freq)

    return pd.DataFrame(flatten(info_queries), columns = ["Query","Word"])
info_queries = format_queries(data_queries, "p")
info_queries

,Query,Word
0,Q1,problem
1,Q1,concern
2,Q1,make
3,Q1,descript
4,Q1,titl
...,...,...
5188,Q112,algorithm
5189,Q112,compar
5190,Q112,previous
5191,Q112,describ


## Appariement

### SRI basé sur le modèle vectoriel avec la fonction Scalar Product

In [14]:
def scalar_product(query,fichier_inverse,info_queries):
    words = list(info_queries.loc[info_queries["Query"] == query, "Word"])
    sri = fichier_inverse.loc[fichier_inverse["Word"].isin(words),["Document","Poid"]]
    sri = sri.groupby('Document')['Poid'].sum().to_frame()
    sri.reset_index(inplace=True)
    sri.rename(columns = {'Poid':'RSV'}, inplace = True)
    sri = sri.sort_values(by=["RSV"])
    return sri.reindex(index=sri.index[::-1])
scalar_product("Q3", fichier_inverse, info_queries)

,Document,RSV
140,D1181,2.870051
172,D1235,2.159867
137,D1179,2.107983
497,D445,1.977708
515,D469,1.655124
...,...,...
639,D62,0.046742
288,D141,0.042847
442,D348,0.039551
563,D53,0.039551


### SRI basé sur le modèle vectoriel avec la fonction Cosine Measure

In [15]:
def cosine_measure(query,fichier_inverse,info_queries):
    words = list(info_queries.loc[info_queries["Query"] == query, "Word"])
    df = fichier_inverse.loc[fichier_inverse["Word"].isin(words),["Document","Poid"]]
    df = df.groupby('Document')['Poid'].sum().to_frame()
    df.reset_index(inplace=True)
    df.rename(columns = {'Poid':'Scalar weight'}, inplace = True)
    df["SQRT weight query"] = np.sqrt(len(words))
    sqrt_weights_doc = fichier_inverse.copy()
    sqrt_weights_doc["Poid"] = np.power(sqrt_weights_doc["Poid"], 2)
    sqrt_weights_doc = sqrt_weights_doc.groupby('Document')['Poid'].sum().to_frame()
    sqrt_weights_doc["Poid"] = np.sqrt(sqrt_weights_doc["Poid"])
    sqrt_weights_doc.rename(columns = {'Poid':'SQRT weight doc'}, inplace = True)
    df = df.merge(sqrt_weights_doc, on="Document")

    df["RSV"] = np.divide(df["Scalar weight"], 
                            np.multiply(df["SQRT weight query"], df["SQRT weight doc"]))

    df = df[["Document", "RSV"]].sort_values(by=["RSV"])
    return df.reindex(index=df.index[::-1])
cosine_measure("Q3", fichier_inverse, info_queries)

,Document,RSV
515,D469,0.494392
140,D1181,0.322355
794,D85,0.306461
623,D599,0.301825
107,D1142,0.290481
...,...,...
183,D1253,0.008899
779,D821,0.008306
551,D512,0.008073
180,D1248,0.007710


### SRI basé sur le modèle vectoriel avec la fonction Jaccard Measure

In [16]:
def jaccard_measure(query,fichier_inverse,info_queries):
    words = list(info_queries.loc[info_queries["Query"] == query, "Word"])
    df = fichier_inverse.loc[fichier_inverse["Word"].isin(words),["Document","Poid"]]
    df = df.groupby('Document')['Poid'].sum().to_frame()
    df.reset_index(inplace=True)
    df.rename(columns = {'Poid':'Scalar weight'}, inplace = True)
    df["square weight query"] = len(words)
    square_weights_doc = fichier_inverse.copy()
    square_weights_doc["Poid"] = np.power(square_weights_doc["Poid"], 2)
    square_weights_doc = square_weights_doc.groupby('Document')['Poid'].sum().to_frame()
    square_weights_doc.rename(columns = {'Poid':'square weight doc'}, inplace = True)
    df = df.merge(square_weights_doc, on="Document")

    df["RSV"] = np.divide(df["Scalar weight"], 
                            np.subtract(np.add(df["square weight query"], df["square weight doc"]), 
                                                df["Scalar weight"]))

    df = df[["Document", "RSV"]].sort_values(by=["RSV"])
    return df.reindex(index=df.index[::-1])
jaccard_measure("Q3", fichier_inverse, info_queries)

,Document,RSV
515,D469,0.296276
794,D85,0.177863
623,D599,0.177654
107,D1142,0.160459
140,D1181,0.159589
...,...,...
795,D850,0.003357
478,D412,0.003001
779,D821,0.002893
692,D687,0.002826


### SRI basé sur le modèle booléen

In [17]:
def format_bool_queries():
    bool_queries = open("../cisi/CISI.BLN").read().replace("\n","")
    bool_queries = bool_queries.replace("\t","")
    bool_queries = bool_queries.split(";")
    del bool_queries[0]
    boolean_queries = []
    for bq in bool_queries:
        if "= " in bq:
            code, bool_query = bq.split("= ")
            boolean_queries.append([code[1:].upper(), bool_query])
    return pd.DataFrame(boolean_queries,columns =["Code","Bool Query"])
bool_queriers = format_bool_queries()
bool_queriers

,Code,Bool Query
0,Q1,"#and ('titles', #or ('automatically', 'retriev..."
1,Q2,"#and (#or('data', 'information'), #..."
2,Q3,"#and ('information',#or ('science', 'definitio..."
3,Q4,"#or (#and ('image', 'recognition'), ..."
4,Q5,"#and (#or ('training', 'use', 'need', 'systems..."
5,Q6,"#and (#or ('communication', 'verbal'), #or (..."
6,Q7,"#and ( #or ('systems', 'data-processing', ..."
7,Q8,"#and (#or ('languages', 'indexing'), #or ('..."
8,Q9,"#and (#or ('information', 'retrieval', 'system..."
9,Q10,"#or (#and ('group', 'mathematics'), ..."


In [18]:
class Node:
    def __init__(self, value=None, children=None):
        self.value = value
        self.children = [Node(c) for c in children] if children else []
    
    def add_children(self, children):
        for child in children:
            self.children.append(Node(child))
    
    def add_child(self, child: str):
        self.children.append(Node(child))
    
    def add_node_child(self, child):
        self.children.append(child)

    def __str__(self):
        return self.value

def get_outside_parenthesis(query):
    # get the first outside parenthesis
    expressions = []
    cpt=0
    opened = False

    for i, c in enumerate(query):

        if c == '(':
            cpt += 1
            if cpt == 1 : 
                start = i
                opened = True

        elif c == ')':
            cpt -= 1

        if cpt == 0 and opened:
            opened = False
            end = i
            expressions.append(query[start+1:end])

    return expressions

def get_tokens(query):
    
    words = []
    operations = []
    temp_query = re.sub(r"\(.+?\)", "", query)
    temp_query = re.sub(r"'", "", temp_query)
    temp_query = re.sub(r"\(", "", temp_query)
    temp_query = re.sub(r"\)", "", temp_query)

    tokens = temp_query.split(',')

    for op in tokens:
        if op.startswith('#'):
            operations.append(op)
        else:
            words.append(op)

    return words, operations

def build_tree(operation, expression):

    words, operations = get_tokens(expression)
    expressions = get_outside_parenthesis(expression)

    todo = list(zip(operations, expressions))

    node = Node(operation)

    for w in words:
        node.add_child(w)

    for op, exp in todo:
        sub_node = build_tree(op, exp)
        node.add_node_child(sub_node)
        
    
    return node

def parcours_tree(root, tv, op):
    result = []
    if type(root) != list : 
        if "#" in root.value:
            if root.value[1:] == "and":
                return parcours_tree(root.children, tv, "and")
            elif root.value[1:] == "or":
                return parcours_tree(root.children, tv, "or")
            elif root.value[1:] == "not":
                return parcours_tree(root.children, tv, "not")
            else:
                return False
    else :
        for child in  root:
            if "#" in child.value:
                if child.value[1:] == "and":
                    if op == "and":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "and")
                        else : result = result & parcours_tree(child.children, tv, "and")
                    elif op == "or":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "and")
                        else : result = result | parcours_tree(child.children, tv, "and")
                    elif op == "not":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "and")
                        else : result = np.revert(parcours_tree(child.children, tv, "and"))
                elif child.value[1:] == "or":
                    if op == "and":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "or")
                        else : result = result & parcours_tree(child.children, tv, "or")
                    elif op == "or":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "or")
                        else : result = result | parcours_tree(child.children, tv, "or")
                    elif op == "not":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "or")
                        else : result = np.revert(parcours_tree(child.children, tv, "or"))
                elif child.value[1:] == "not":
                    if op == "and":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "not")
                        else : result = result & parcours_tree(child.children, tv, "not")
                    elif op == "or":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "not")
                        else : result = result | parcours_tree(child.children, tv, "not")
                    elif op == "not":
                        if len(result) == 0: result = parcours_tree(child.children, tv, "not")
                        else : result = np.revert(parcours_tree(child.children, tv, "not"))
                else:
                    return False
            else :
                if op == "and":
                    if len(result) == 0: result = tv[child.value]
                    else : result = result & tv[child.value]
                elif op == "or":
                    if len(result) == 0: 
                        result = tv[child.value]
                    else : 
                        result = result | tv[child.value]
                elif op == "not":
                    result = np.invert(tv[child.value])
                else :
                    return False
        return result

def boolean_model(query, fichier_inverse, info_queries, stem):
    q1 = info_queries.loc[info_queries["Code"] == "Q1", "Bool Query"][0]
    q2 = q1.split("#")
    del q2[0]
    rgx = '(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%? |\w+(?:\-\w+)*'
    text_token = nltk.RegexpTokenizer(rgx).tokenize(" ".join(q2))
    mots_vides = nltk.corpus.stopwords.words('english')
    words_of_query = [terme for terme in text_token if terme.lower() not in mots_vides]
    ps = nltk.PorterStemmer()
    ls = nltk.LancasterStemmer()
    if stem == "p": stemmed = [ps.stem(tt) for tt in words_of_query]
    elif stem == "l": stemmed = [ls.stem(tt) for tt in words_of_query]
    tv = []
    for d in fichier_inverse["Document"].unique():
        tv_row = list(np.where(np.isin(stemmed, fichier_inverse.loc[fichier_inverse["Document"] == d, "Word"].to_numpy()),True,False))
        tv_row.insert(0,d)
        tv.append(tv_row)
    words_of_query.insert(0,"Document")
    table_verite = pd.DataFrame(tv, columns = words_of_query)
    table_verite = table_verite.sort_values(by=["Document"])
    query = re.sub("\s", "", q1)
    query = re.sub("#q\d+?=", "", query)
    operation = re.match(r"#\w+", query).group(0)
    expression = query[len(operation)+1: -1]

    root = build_tree(operation, expression)

    result =  parcours_tree(root, table_verite, "")
    table_verite["Result"] = result.astype(int)
    return table_verite[["Document", "Result"]]
tv = boolean_model("Q1", fichier_inverse, bool_queriers,"p")
tv = tv.sort_values(by=["Result"])
tv = tv.reindex(index=tv.index[::-1])
tv

,Document,Result
24,D1091,1
0,D1090,1
652,D1421,1
1076,D1230,1
317,D1118,1
...,...,...
1215,D540,0
1363,D542,0
162,D543,0
407,D544,0


### SRI basé sur le modèle probabiliste avec la fonction BM25

In [19]:
def bm25(query, fichier_inverse, info_queries, B, K):
    N = len(fichier_inverse['Document'].unique())
    words = list(info_queries.loc[info_queries["Query"] == query, "Word"])
    df = fichier_inverse.loc[fichier_inverse["Word"].isin(words)]
    nb_doc = df.groupby('Word')['Document'].count().to_frame()
    nb_doc.reset_index(inplace=True)
    nb_doc.rename(columns = {'Document':'Nombre document contenu'}, inplace = True)
    df = df.merge(nb_doc, on="Word")
    nb_termes_doc = fichier_inverse.groupby('Document')['Word'].count().to_frame()
    nb_termes_doc.reset_index(inplace=True)
    nb_termes_doc.rename(columns = {'Word':'dl'}, inplace = True)
    df = df.merge(nb_termes_doc, on="Document")
    df["avdl"]= nb_termes_doc["dl"].mean()
    rsv_list = []
    for d in df["Document"].unique():
        temp = df.loc[df["Document"] == d, ["Frequence", "Nombre document contenu", "dl", "avdl"]]
        ni = temp["Nombre document contenu"]
        rsv = np.multiply(np.divide(temp["Frequence"], 
                            np.add(np.multiply(K, 
                            np.add(np.subtract(1, B), 
                                    np.multiply(B, np.divide(temp["dl"], temp["avdl"])))), 
                                                temp["Frequence"])), 
                                                np.log10(np.divide(np.add(
                                                        np.subtract(N, ni), 0.5), np.add(ni, 0.5))))
        
        rsv_list.append([d, np.sum(rsv)])
    df = pd.DataFrame(rsv_list, columns = ["Document","RSV"])
    df = df.sort_values(by=["RSV"])
    return df.reindex(index=df.index[::-1])
bm25("Q1", fichier_inverse, info_queries, 0.6, 1.5)



,Document,RSV
200,D1419,2.523655
168,D429,2.493753
202,D489,2.470111
11,D65,2.381813
257,D666,2.374978
...,...,...
723,D593,0.185210
727,D264,0.184179
764,D158,0.184179
731,D223,0.174469


### SRI basé sur le modèle de datamining DBSCAN (Clustering)

### SRI basé sur le modèle de datamining Naïve Bayes (Classification)